In [1]:
import numpy as np
import pandas as pd
import requests

from PB_funciones import nuevo_libro, nuevo_libro_titulo

In [2]:
# ${{secrets.APIKEY_ISBN}} #----------> No funcionan los secretos!!!

In [3]:
# Se crea el diccionario que contendrá los valores de los libros solicitados.
dict_books = {}


In [4]:
# Web scraping de la página de Wikipedia con los libros más vendidos (español)
url = "https://es.wikipedia.org/wiki/Anexo:Libros_m%C3%A1s_vendidos"
respuesta = requests.get(url)
tablas_libros = pd.read_html(respuesta.content, encoding = "utf8")

df_libros = pd.DataFrame()
df_libros.drop(df_libros.index, inplace = True)
for i in range(len(tablas_libros)-1):
    if i == 0:
        df_libros = pd.DataFrame(tablas_libros[i])
    else:
        df_libros = pd.concat([df_libros,pd.DataFrame(tablas_libros[i])], axis = 0)

df_libros

,Libro,Autor,Idioma original,Primera edición,Ventas aproximadas
0,Don Quijote,Miguel de Cervantes,Español,1605,Más de 500 millones
1,Historia de dos ciudades (A Tale of Two Cities),Charles Dickens,Inglés,1859,Más de 200 millones[3]​
2,El Señor de los Anillos (The Lord of the Rings),J. R. R. Tolkien,Inglés,1954,150 millones[4]​[5]​
3,Harry Potter y la piedra filosofal (Harry Pott...,J.K. Rowling,Inglés,1997,140 millones[6]​
4,El principito (Le Petit Prince),Antoine de Saint-Exupéry,Francés,1943,140 millones[7]​
...,...,...,...,...,...
44,Observaciones de Yu Dan sobre las Analectas de...,Yu Dan,Chino,2006,10 millones[106]​
45,La mujer total (The Total Woman),Marabel Morgan,Inglés,1974,10 millones[107]​
46,"Historia del futuro, la sociedad del conocimie...",Taichi Sakaiya,Japonés,1985,10 millones[108]​
47,Problemas de la economía socialista de China (...,Xue Muqiao,Chino,1979,10 millones[109]​


In [5]:
# Web scraping de la página de Wikipedia con los libros más vendidos (inglés)
url = "https://en.wikipedia.org/wiki/List_of_best-selling_books"
respuesta = requests.get(url)
tablas_libros = pd.read_html(respuesta.content, encoding = "utf8")

df_books = pd.DataFrame()
df1 = pd.DataFrame(tablas_libros[0])
df2 = pd.DataFrame(tablas_libros[1])
df3 = pd.DataFrame(tablas_libros[2])
df4 = pd.DataFrame(tablas_libros[3])
df_books = pd.concat([df1,df2,df3,df4], axis = 0)

df_books

,Book,Author(s),Original language,First published,Approximate sales,Genre,Approx. sales
0,A Tale of Two Cities,Charles Dickens,English,1859,>200 million[21],Historical fiction,NaN
1,The Little Prince (Le Petit Prince),Antoine de Saint-Exupéry,French,1943,200 million[22][23][24],"Fantasy, children's fiction",NaN
2,The Alchemist (O Alquimista),Paulo Coelho,Portuguese,1988,150 million[25][26],Fantasy,NaN
3,Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120 million[27][28],"Fantasy, children's fiction",NaN
4,And Then There Were None,Agatha Christie,English,1939,100 million[29],Mystery,NaN
...,...,...,...,...,...,...,...
64,Fahrenheit 451,Ray Bradbury,English,1953,NaN,NaN,10 million[188]
65,Angela's Ashes,Frank McCourt,English,1996,NaN,NaN,10 million[189]
66,The Story of My Experiments with Truth (સત્યના...,Mohandas Karamchand Gandhi,Gujarati,1925–1929,NaN,NaN,10 million[190]
67,Bridget Jones's Diary,Helen Fielding,English,1996,NaN,NaN,10 million (as of 2005)[191]


title = "Don Quijote"
h = {"Authorization": "61066_f9ba2c2c66284291bfbe3153a53fdb85"} # ...............................Borrar API en entrega final!!!
prueba = requests.get(f"https://api2.isbndb.com/books/{title}?page=1&pageSize=1&column=title&shouldMatchAll=1", headers= h)

dicc = {}
book_new = prueba.json().get("books", {})
dicc.update({
    "ISBN": book_new[0].get("isbn13", ""),
    "Título": book_new[0].get("title", ""),
    "Autor": ", ".join(book_new[0].get("authors", [])),
    "Editorial": book_new[0].get("publisher", ""),
    "Páginas": book_new[0].get("pages", ""),
    "Fecha de publicación": book_new[0].get("date_published", ""),
    #"Subjects": ", ".join(book_info.get("subjects", [])),
    #"Binding": book_info.get("binding", ""),
    "Sinopsis": book_new[0].get("synopsis", ""),
    "Lengua original": book_new[0].get("language", ""),
    "Edición": book_new[0].get("edition", ""),
    "Dimensiones": book_new[0].get("dimensions", ""),
    #"MSRP": book_info.get("msrp", ""),
    #"Image": book_info.get("image", ""),
})

dicc
#print(book_new)

In [6]:
df_books.reset_index(inplace = True)

In [7]:
titulo_rect = df_books["Book"]
titulo_rect[1]

'The Little Prince (Le Petit Prince)'

In [8]:
indice = titulo_rect[1].index("(")
titulo = titulo_rect[1]
nuevo_titulo = titulo[:indice]
titulo_fin = nuevo_titulo.strip()
titulo_fin


'The Little Prince'

In [12]:
diccionario_test = {}
nuevo_libro(diccionario_test, 9780156465113)
diccionario_test

UnboundLocalError: cannot access local variable 'respuesta' where it is not associated with a value

In [11]:
test = 9780156465113
if type(test) == int:
    print("Hola")

Hola
